# Lab 5/Labo 5

This lab consists of 2 parts:
* Understanding linear and kernel perceptrons
* Implementation of linear and kernel perceptrons
 
 <hr>
 
Ce lab consiste en 2 parties:
* Comprendre le perceptron linéaire et le perceptron à noyau
* Implémenter le perceptron linéaire et le perceptron à noyau

## Perceptron and Stochastic Gradient Descent/Perceptron et descente du gradient stochastique

We have studied that given a model and a cost function, we learn the parameters of the model by using stochastic gradient descent. This consists in iteratively modifying the parameters by looping on the examples $\left(x, y\right)$ and by adding a gradient term.

<hr>

Nous avons étudié que considérant un modèle et une fonction de coût, il est possible d'apprendre les paramètres d'un modèle avec la descente du gradient stochastique. Ceci consiste à mettre à jour de manière itérative les paramètres en passant au travers tous les exemples $\left(x, y\right)$ et en leur ajoutant le gradient calculé.





### Perceptron Algorithm/L'algorithme perceptron

The perceptron algorithm is a stochastic learning algorithm that iteratively improves a model by running it on training samples, then updating the model whenever it finds it has made an incorrect classification. The model learned by the standard perceptron algorithm is a linear classifier: a vector of weights $\mathbf{w}$ that is used to classify a sample vector $\mathbf{x}$ as class $1$ or class $-1$ according to

$${{\hat {y}}=\operatorname {sgn} (\mathbf {w} ^{\top }\mathbf {x} )}$$
where a zero is mapped to minus one. (The "hat" on $\hat{y}$ denotes an estimated value.)

In pseudocode, the perceptron algorithm is given by:

* Initialize $\mathbf{w}$ to an all-zero vector of length $p$, the number of predictors (features).
* For some fixed number of iterations, or until some stopping criterion is met:
    - For each training example $\mathbf{x}_i$ with ground truth label $y_i ∈ \{-1, 1\}$:

    Let, $\hat{y} = sgn(\mathbf{w}^T \mathbf{x}_i)$
    
    If $ \hat{y} \neq y_i$, $\mathbf{w} := \mathbf{w} + y_i \mathbf{x}_i$
    
<hr>

Le perceptron est un algorithme stochastique qui apprend, à l'aide d'exemples de d'entraînement, à mieux classifier en mettant à jour les paramètres dès qu'une erreur de classification est commise. L'algorithme perceptron standard est un classifieur linéaire. Plus précisément, il apprend un vecteur de poids $\mathbf{w}$ qui est utilisé pour classifier un vecteur $\mathbf{x}$ comme appartenant à la classe $1$ ou $-1$ selon 

$${{\hat {y}}=\operatorname {sgn} (\mathbf {w} ^{\top }\mathbf {x} )}$$
où un zéro sera attribué à la classe $-1$.

En pseudo-code, l'algorithme du perceptron opère comme suit:

* Initialise $\mathbf{w}$ en un vecteur de zéros de dimension $p$, où $p$ est le nombre de traits.
* Pour un nombre fixe d'itérations ou jusqu'à ce qu'un critère soit rencontré:
    - Pour chaque exemple d'entraînement $\mathbf{x}_i$ avec la vraie classe $y_i ∈ \{-1, 1\}$:

    Soit, $\hat{y} = sgn(\mathbf{w}^T \mathbf{x}_i)$
    
    Si $ \hat{y} \neq y_i$, $\mathbf{w} := \mathbf{w} + y_i \mathbf{x}_i$

## Part 2: Non-Linear Transformations/Partie 2: transformations non-linéaires

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import random
import time

In [0]:
! rm cercle.txt ellipse.txt
! wget https://drive.google.com/open?id=1UNh_lzZdW9ePeyDvpGdcw0QURk23Ny3i
#! wget https://www.dropbox.com/s/chfkj30rlnez38o/cercle.txt
! wget https://www.dropbox.com/s/u0jxsd8a0iuiqgx/ellipse.txt

--2019-10-16 16:10:11--  https://drive.google.com/open?id=1UNh_lzZdW9ePeyDvpGdcw0QURk23Ny3i
Resolving drive.google.com (drive.google.com)... 172.217.203.101, 172.217.203.138, 172.217.203.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.101|:443... connected.
HTTP request sent, awaiting response... 307 OK
Location: https://drive.google.com/file/d/1UNh_lzZdW9ePeyDvpGdcw0QURk23Ny3i/view?usp=drive_open [following]
--2019-10-16 16:10:11--  https://drive.google.com/file/d/1UNh_lzZdW9ePeyDvpGdcw0QURk23Ny3i/view?usp=drive_open
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘open?id=1UNh_lzZdW9ePeyDvpGdcw0QURk23Ny3i’

open?id=1UNh_lzZdW9     [ <=>                ]  66.37K  --.-KB/s    in 0.002s  

2019-10-16 16:10:11 (30.1 MB/s) - ‘open?id=1UNh_lzZdW9ePeyDvpGdcw0QURk23Ny3i’ saved [67966]

--2019-10-16 16:10:12--  https://www.dropbox.com/s/u0jxsd8a0iuiqgx/ellipse.txt
Resolving ww

In [0]:
# We start by loading the dataset
# Nous commencons par télécharger le dataset

data = np.loadtxt('cercle.txt')
# data = np.loadtxt('ellipse.txt')

# There are only 2 dimensions...
# Il n'y a que 2 dimensions...
train_cols = [0, 1]
# A variable to hold the index of the label column.
# On définit l'index de la colonne représentant les classes.
target_ind = [data.shape[1] - 1]

n_classes = 2
n_train = 1500

# Comment to get non-deterministic results
# Mettre en commentaire pour avoir des résultats non-déterministes
random.seed(3395)

# Randomly select indices for the training and test set
# On sélectionne aléatoirement les indices and les train et test set
inds = np.arange(data.shape[0])
random.shuffle(inds)
train_inds = inds[:n_train]
test_inds = inds[n_train:]
    
# Separate the dataset into two sets: training and test.
# On sépare le dataset en training et test set.
train_set = data[train_inds,:]	
train_set = train_set[:, train_cols + target_ind] 
test_set = data[test_inds,:]
test_set = test_set[:, train_cols + target_ind]

# Separate the test set: inputs and labels.
# On sépare le test set en entrées et en étiquettes
test_inputs = test_set[:,:-1]
test_labels = test_set[:,-1]

# Function to plot the decision boundary
# Fonction pour tracer la frontière de décision
def plot(model, outputs):
    train_classes_pred = np.sign(outputs)
    plt.scatter(train_set[:, 0], train_set[:, 1], c=train_classes_pred)
    plt.scatter(test_set[:, 0], test_set[:, 1], c=classes_pred)
    plt.title('Prediction')
    plt.show()

    plt.scatter(train_set[:, 0], train_set[:, 1], c=train_set[:,-1])
    plt.scatter(test_set[:, 0], test_set[:, 1], c=test_labels)
    plt.title('True labels')
    plt.show()

OSError: ignored

To demonstrate the use of non-linear transformations, we will make use of the circle and ellipse datasets, available on the website. We will first apply a transformation on the dataset before training. The algorithm will then uniquely work with the transformed data. To keep things simple at first, we will simply implement a transformation consisting of a degree two polynomial. And to make it even more simple, the circle and ellipse datasets are only in 2-d...

$$\phi : \mathbb{R}^2 \rightarrow \mathbb{R}^6$$

$$\phi(x_1, x_2) = \left(1,x_1,x_2,x_1^2,x_2^2,x_1x_2\right)$$

**Exercise: implement this transformation in the polynomial function below:**

<hr>

Afin de démontrer l'utilité des transformations non-linéaires, nous allons faire usage des jeux de données *circle* et *ellipse*, disponible sur le lien fourni. Nous appliquerons une transformation sur le jeu de données avant d'entraîner nos modèles. L'algorithme sera ainsi entrâiner uniquement avec les données transformées. Pour simplifier les choses de prime abord, nous implémenterons une transformation polynomiale de degré 2. De plus, les jeux de données *circle* et *ellipse* n'ont que 2 dimensions. 

$$\phi : \mathbb{R}^2 \rightarrow \mathbb{R}^6$$

$$\phi(x_1, x_2) = \left(1,x_1,x_2,x_1^2,x_2^2,x_1x_2\right)$$

**Exercice: implémentez cette transformation à l'intérieur de la fonction polynomiale ci-dessous:**

In [0]:
# Takes a matrix of data examples x (n datapoints, input space dimension) as input (without labels) and returns the transformed
# matrix (n datapoints, projected space dimension)
# Cette fonction prend une matrice d'exemples x de dimension (n nombre de points, nombre de traits) comme entrée (sans les étiquettes)
# et retourne la matrice transformée de dimension (n nombre de points, dimension de l'espace projeté)
def polynomial(X):
    Y = np.zeros((X.shape[0], 6))
    Y[:, 0] = 1.
    # Please fill in the code below/complétez le code ci-dessous
    Y[:, 1] = ...
    Y[:, 2] = ...
    Y[:, 3] = ...
    Y[:, 4] = ...
    Y[:, 5] = ...
    return Y

**Exercise: Now that the transformation is implemented, you should complete the `train` method inside the Perceptron class:**

<hr>

**Exercice: Maintenant que la transformation est effectuée, complétez la méthode `train` de la classe Perceptron:**

In [0]:
class Perceptron:
    def __init__(self):
        return

    def train(self, train_data):
        n_example = train_data.shape[0]            
        self.weights = np.random.random(train_data.shape[1] - 1)

        i = 0
        count = 0 # We stop when the set is linearly separated/On arrête lorsque le dataset est séparé linéairement
        n_iter = 0
        n_iter_max = n_example * 100
        while (count < n_example and n_iter < n_iter_max):
            # Please fill in the code below/Complétez le code ci-dessous
            if ...:
                self.weights = ...
                count = 0
            else:
                count = count + 1
            i = (i + 1) % n_example
            n_iter += 1

    def compute_predictions(self, test_data):
        outputs = []
        for data in test_data:
            outputs.append(np.dot(data, self.weights))
        return outputs

In [0]:
print("We will train a perceptron on ", n_train, " training examples")

# Transform the dataset
# On transforme le jeu de données
transformed_train_set = np.concatenate((polynomial(train_set[:,:-1]), train_set[:, -1][:, None]), axis=1)
transformed_test_inputs = polynomial(test_inputs)

# Create and train the model
# On crée une instance et l'on entraîne le modèle
model = Perceptron()
model.train(transformed_train_set)

# Obtain the ouputs on the test set.
# On obtient les sorties du test set
outputs = model.compute_predictions(transformed_test_inputs)

# Convert the outputs into classes by taking the sign.
# On convertit les sorties en classes en effectuant la fonction signe 
classes_pred = np.sign(outputs)

# Measure the performance
# On mesure la performance
err = 1.0 - np.mean(test_labels==classes_pred)
print("The test error is ", 100.0 * err,"%")

# Trace the decision boundary
# On trace la frontière de décision
outputs = model.compute_predictions(transformed_train_set[:,:-1])
plot(model, outputs)

We will train a perceptron on  1500  training examples


TypeError: ignored

## 2. Kernel trick/L'astuce du noyau

Your objective for this section is to implement a gaussian kernel perceptron. We will ask you to use a gaussian kernel for this problem. For a review, please refer to the [course document](https://studium.umontreal.ca/pluginfile.php/4043631/mod_resource/content/2/13_kernel_trick-en-3395.pdf).

**Exercise: implement a polynomial kernel function of arbitrary degree**

<hr>

Votre but dans cette section est d'implémenter un noyau gaussien pour le perceptron. Nous allons utilisé le noyau Gaussien pour ce problème. Pour une révision de l'astuce du noyau, référez-vous aux [notes de cours](https://studium.umontreal.ca/pluginfile.php/5052299/mod_resource/content/1/13_kernel_trick.pdf).

**Exercice: implémentez une fonction de noyau polynomiale de degré arbitraire**

In [0]:
def kernel_polynomial(x, y, deg=2):
    # https://en.wikipedia.org/wiki/Polynomial_kernel
    return ...

def kernel_rbf(x, y):
    return ...

### Kernel methods

A *kernel machine* is a classifier that stores a subset of its training examples $\mathbf{x}_i$, associates with each a weight $α_i$, and makes decisions for new samples $\mathbf{x}'$ by evaluating

$${\operatorname{sgn} \sum _{i=1}^N\alpha _{i}y_{i}K(\mathbf {x} _{i},\mathbf {x'} )}$$

Here, $K$ is some kernel function. Formally, a kernel function is a non-negative semidefinite kernel, representing an inner product between samples in a high-dimensional space, as if the samples had been expanded to include additional features by a function $\phi: K(\mathbf{x}, \mathbf{x'}) = \phi(\mathbf{x})^T \phi(\mathbf{x}')$. Intuitively, it can be thought of as a similarity function between samples.

### Kernel Perceptron

To derive the kernel perceptron algorithm, we must first formulate it in dual form, starting from the observation that the weight vector $\mathbf{w}$ can be expressed as a linear combination of the $n$ training samples. The equation for the weight vector is
$$\mathbf{w} =\sum _{i}^{n}\alpha _{i}y_{i}\mathbf {x} _{i}$$
where $\alpha_i$ is the number of times $\mathbf{x}_i$ was misclassified, forcing an update $\mathbf{w} := \mathbf{w} + y_i x_i$. Using this result, we can formulate the dual perceptron algorithm, which loops through the samples as before, making predictions, but instead of storing and updating a weight vector $\mathbf{w}$, it updates a *mistake counter* vector $\mathbf{\alpha}$. We must also rewrite the prediction formula to get rid of $\mathbf{w}$:

$${\begin{aligned}{\hat{y}}&=\operatorname {sgn}(\mathbf {w} ^{\mathsf {T}}\mathbf {x} )\\&=\operatorname {sgn} \left(\sum _{i}^{n}\alpha _{i}y_{i}\mathbf {x} _{i}\right)^{\mathsf {T}}\mathbf {x} \\&=\operatorname {sgn} \sum _{i}^{n}\alpha _{i}y_{i}(\mathbf {x} _{i}\cdot \mathbf {x} )\end{aligned}}$$


Plugging these two equations into the training loop turn it into the dual perceptron algorithm.

Finally, we can replace the dot product in the dual perceptron by an arbitrary kernel function, to get the effect of a feature map $\phi$ without computing $\phi(x)$ explicitly for any samples. Doing this yields the kernel perceptron algorithm:

* Initialize $\alpha$ to an all-zeros vector of length $n$, the number of training samples.
* For some fixed number of iterations, or until some stopping criterion is met:
    For each training example $x_j$, $y_j$:
    Let $${\hat{y}}=\operatorname {sgn} \sum _{i}^{n}\alpha _{i}y_{i}K(\mathbf {x} _{i},\mathbf {x} _{j})$$
    If $\hat{y} \neq y_j$, perform an update by incrementing the mistake counter:
    $$\alpha_j := \alpha_j + 1$$
    
<hr>

### Les méthodes à noyau

Une machine à noyau est un classifieur qui enregistre une partie des exemples d'entraînement $\mathbf{x}_i$, qui associe ces exemples à un poids $α_i$ et qui prend une décision pour de nouveaux exemples en évaluant:

$${\operatorname{sgn} \sum _{i=1}^N\alpha _{i}y_{i}K(\mathbf {x} _{i},\mathbf {x'} )}$$

Ici, $K$ est une fonction de noyau quelconque. Formellement, une fonction de noyau est semi-définie positive, représentant un produit scalaire dans l'espace à haute dimension où les exemples sont *mappés*. En d'autres mots, les exemples sont *mappés* vers un espace à plus haute dimension par la fonction $\phi: K(\mathbf{x}, \mathbf{x'}) = \phi(\mathbf{x})^T \phi(\mathbf{x}')$, de manière à ce que ces exemples contiennent de nouveaux traits. Intuitivement, ceci peut être vu comme étant une fonction de ressemblance entre les exemples.

### Le perceptron à noyau

Afin de dériver le perceptron à noyau, nous devons premièrement le formuler dans ses deux formes, en commencant par exprimer le vecteur de poids $\mathbf{w}$ par une combinaison linéaire des $n$ exemples d'entraînement. L'expression des poids $\mathbf{w}$ se veut comme suit:

$$\mathbf{w} =\sum _{i}^{n}\alpha _{i}y_{i}\mathbf {x} _{i}$$
où $\alpha_i$ est le nombre de fois que $\mathbf{x}_i$ a été mal classé, imposant une mise à jour $\mathbf{w} := \mathbf{w} + y_i x_i$. En utilisant cette équation, nous pouvons maintenant reformuler l'algorithme du preceptron, qui passent à travers les exemples, comme précédemment, en effectuant des predictions mais, plutôt que d'emmagasiner et de mettre à jour le vecteur de poids $\mathbf{w}$, nous mettrons à jour le *compteur* d'erreur $\mathbf{\alpha}$. Ainsi, pour se débarasser du terme $\mathbf{w}$, nous réécrivons la formule de prédiction comme suit:

$${\begin{aligned}{\hat{y}}&=\operatorname {sgn}(\mathbf {w} ^{\mathsf {T}}\mathbf {x} )\\&=\operatorname {sgn} \left(\sum _{i}^{n}\alpha _{i}y_{i}\mathbf {x} _{i}\right)^{\mathsf {T}}\mathbf {x} \\&=\operatorname {sgn} \sum _{i}^{n}\alpha _{i}y_{i}(\mathbf {x} _{i}\cdot \mathbf {x} )\end{aligned}}$$

Insérez ces deux expressions dans la méthode d'entraînement résulte à l'algorithme du perceptron duel.

Finalement, nous pouvons remplacer le produit scalaire par une fonction de noyau quelconque afin d'avoir un *mapping* $\phi$ sans avoir à explicitement calculer la transformation $\phi(x)$ pour les exemples. Effectuer cette boucle est par définition l'algorithme du perceptron à noyau:  

* Initialise $\alpha$ par un vecteur de zéros de dimension $n$, soit le nombre d'exemples d'entraînement.
* Pour un nombre fixe d'itérations ou jusqu'à ce qu'un critère soit rempli:
    Pour chaque exemple d'entraînement $x_j$, $y_j$:
    Soit $${\hat{y}}=\operatorname {sgn} \sum _{i}^{n}\alpha _{i}y_{i}K(\mathbf {x} _{i},\mathbf {x} _{j})$$
    Si $\hat{y} \neq y_j$, effectue une mise à jour en ajoutant 1 au *compteur* d'erreurs:
    $$\alpha_j := \alpha_j + 1$$





Now, we will implement the kernel perceptron.

**Exercise: Complete the code below for the kernel perceptron algorithmn**

Hint: Reuse the perceptron code above, but rather than learning the vector $w$ directly, you need to rewrite $\mathbf{w} = \sum_i \alpha_i \mathbf{x}_i$ and to learn the coefficients $a_i$. Then, we replace $\mathbf{w}^T x$ by $\left(\sum_i \alpha_i \mathbf{x}_i\right)^T x$ and apply the kernel trick.

<hr>

Maintenant, nous allons implémenter le perceptron à noyau.

**Exercice: Complétez le code suivant pour l'algorithme du perceptron avec noyau**

Indication: Repartez du code du perceptron ci-dessus, mais plutôt que d'apprendre le vecteur $w$ directement, il faut le réécrire $w = \sum_i \alpha_i x_i$ et apprendre les coefficients $a_i$. Ensuite, on remplace $w^T x$ par $\left(\sum_i \alpha_i x_i\right)^T x$ puis, on applique l'astuce du noyau.

In [0]:
class KernelPerceptron:
    def __init__(self, kernel_fn):
        self.kernel_fn = kernel_fn

    def train(self, train_data):
        n_example = train_data.shape[0]
        self.train_data = np.array(train_data)

        # alpha initialization
        # initialisation d'alpha
        self.a = np.zeros(n_example)

        # Get rid of the labels
        # On enlève les étiquettes
        train_x = self.train_data[:,:-1]
        train_y = self.train_data[:,-1]

        self.train_x = train_x
        self.train_y = train_y

        # Gram matrix
        K = np.zeros((n_example, n_example))
        for i in range(n_example):
            K[i] = self.kernel_fn(train_x[i], train_x)

        # Kernel calculation
        # Calcul du noyau
        i = 0
        count = 0
        n_iter = 0
        n_iter_max = n_example * 100
        while (count < n_example and n_iter < n_iter_max):
            # Please write in the following code/écrivez votre code ici
            if ...:
                ...
                count = 0
            else:
                count = count + 1
            i = (i + 1) % n_example
            n_iter += 1

    def compute_predictions(self, test_data):
        outputs = []
        for i in range(len(test_data)):
            # Please write in the following code/écrivez votre code ici
            prediction = ...
            outputs.append(prediction)
        return outputs

In [0]:
print("We will learn a kernel perceptron on ", n_train, " training examples")

model = KernelPerceptron(kernel_polynomial)
model.train(train_set)

outputs = model.compute_predictions(test_inputs)
classes_pred = np.sign(outputs)
err = 1.0 - np.mean(test_labels==classes_pred)
print("The test error is {}%".format(100.0 * err))
outputs = model.compute_predictions(train_set[:,:-1])

plot(model, outputs)